In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.optim import lr_scheduler
from torch.optim import lr_scheduler
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import random_split

import math
import numpy as np
from scipy import stats
def rsquared(x, y): 
    _, _, r_value, _, _ = stats.linregress(x.detach().cpu().numpy(), y.detach().cpu().numpy()) 
    return r_value**2


def normalization(x: list):
    M, m = np.max(x), np.min(x)
    for i in range(len(x)):
        x[i] = (x[i] - (M + m) / 2) / ((M - m) / 2)
    # x in [-1, 1]
    return M, m, x
def cal(x: list):
    mu, sigma = 0, 0
    for i in range(len(x)):
        mu += x[i]
    mu /= len(x)*1.0
    for i in range(len(x)):
        sigma += (x[i]-mu) ** 2
    sigma /= len(x)*1.0
    from math import sqrt
    for i in range(len(x)):
        x[i] = (x[i]-mu) / sqrt(sigma)
    return mu, sigma, x

def ArrNorm(x: np.ndarray, state='max-min'):
    assert isinstance(x, np.ndarray), "We need a list"
    
    if state == 'max-min':
        M_list, m_list, res = [], [], []
        for i in range(x.shape[0]):
            u = x[i].tolist()
            M, m, t = normalization(u)
            res.append(t)
            M_list.append(M)
            m_list.append(m)
        return M_list, m_list, np.array(res)
    elif state == 'standard':
        mu_list, sigma_list, res = [], [], []
        for i in range(x.shape[0]):
            u = x[i].tolist()
            m, s, t = cal(u)
            res.append(t)
            mu_list.append(m)
            sigma_list.append(s)
        return mu_list, sigma_list, res


def df2arr(x) -> np.ndarray:
    return np.array(x, dtype=np.float32)


In [ ]:
class NET(nn.Module):
    def __init__(self, seq, batch_size, ablation_scale=1.0):
        super(NET, self).__init__()
        self.seq = seq
        self.batch_size = batch_size
        self.scale = ablation_scale
        self.input_size = self.hidden_size = seq
        
        self.is_directional = False
        self.num_direction = 2 if self.is_directional else 1
        self.num_layers = 1
        
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, dropout=0.5, bidirectional=self.is_directional, batch_first=False)
        
        self.decouple = nn.Sequential(
            nn.Linear(self.seq*5,self.seq*5),  # linear decouple
            nn.Dropout(0.5),
#             nn.Sigmoid(),   # use sigmoid to enhance unlinear regression may cause data shifting
            nn.Linear(self.seq*5, self.seq*5),     # unlinear pool
        )
        self.conv = nn.Sequential(
            # seq * 5 
            nn.Conv2d(1, 2, kernel_size=(3,3), padding=1, bias=False), # seq * 5
            nn.BatchNorm2d(2),
            nn.ReLU(inplace=True),

            nn.Conv2d(2, 2, kernel_size=(1,1), padding=0, bias=False),  # seq * 5
            nn.BatchNorm2d(2),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(2, 1, kernel_size=(3,3), padding=1, bias=True), # seq * 5
            nn.ReLU(inplace=True),
        )
    
    def reset(self, scale=1.0):
        self.ablatiion_scale = scale

    def forward(self, x):
        x = x.squeeze()
        
        h_0 = torch.randn(self.num_direction*self.num_layers, self.seq, self.hidden_size).cuda()
        c_0 = torch.randn(self.num_direction*self.num_layers, self.seq, self.hidden_size).cuda()
        pred, *_ = self.lstm(x, (h_0, c_0))
        pred = pred.flatten(1)
        out1 = self.decouple(pred)
        with torch.no_grad():
            out1 = out1.reshape(-1, 1, self.seq, 5)
        out2 = self.conv(out1)
        assert out1.shape==out2.shape, "Shape Unequal Error."
        return self.scale * out1 + out2

In [ ]:
excel = pd.read_excel('/kaggle/input/a32-data/A32.xlsx', header=None)
excel.shape

sp = [1486, 2972, 4458]
station_1 = excel.iloc[1:sp[0]+1,1:6]
station_2 = excel.iloc[sp[0]+1:sp[1]+1,1:6]
standard = excel.iloc[sp[1]+1:sp[2]+1,1:6]

station_1 = df2arr(station_1)
station_2 = df2arr(station_2)
standard = df2arr(standard)
station_1.shape, station_2.shape, standard.shape

In [ ]:
s1_minus_sd = station_1 - standard
s2_minus_sd = station_2 - standard

In [ ]:
s1_M, s1_m, s1 = ArrNorm(station_1, 'standard')
s2_M, s2_m, s2 = ArrNorm(station_2, 'standard')

In [ ]:
def GetDataset(input_arr: list, output_arr: list, seq: int):
    assert(len(input_arr)==len(output_arr)), "Different size of input and output!"
    Input = []
    Output = []
    for i in range(len(input_arr)-seq):
        Input.append(input_arr[i:i+seq][:])
        Output.append(output_arr[i:i+seq][:])
    return torch.tensor(Input, dtype=torch.float32), torch.tensor(Output, dtype=torch.float32)

        
def load_array(data_arrays, batch_size, is_train=True):
    # data-iter
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


s1_minus_sd.shape

In [ ]:
Input_Data_1, Output_Data_1 = GetDataset(s1, s1_minus_sd, sequence) 
Input_Data_2, Output_Data_2 = GetDataset(s2, s2_minus_sd, sequence)

In [ ]:
sequence = 5
batch_size = 32     # or 16
base_scale = 6.0

pth_path = ''

Test_Iter_1 = load_array((Input_Data_1, Output_Data_1), batch_size=Input_Data_1.shape[0], shuffle=True, drop_last=True)
Test_Iter_2 = load_array((Input_Data_2, Output_Data_2), batch_size=Input_Data_2.shape[0], shuffle=True, drop_last=True)

model = NET(batch_size=batch_size, seq=sequence)
model.load_state_dict(torch.load(pth_path))
model = model.cuda()
model.eval()

In [ ]:


R1, R2 = [], []
scale1, scale2 = [], []
r_1, r_2 = .0, .0
scale_1, scale_2 = 0, 0

for idx in range(5):
    for scale in np.arange(base_scale-5., base_scale+5., .5):
        model.reset(scale=scale)
        with torch.no_grad():
            for i, use in enumerate(Test_Iter_1):
                pred = model(use[0].cuda())
                pred = pred.reshape(-1, 5)
                use[1] = use[1].reshape(-1, 5)
                assert pred.shape == use[1].shape, 'unequal shape error'
                r1 = rsquared(pred[idx], use[1][idx])
            for i, use in enumerate(Test_Iter_2):
                    pred = model(use[0].cuda())
                    pred = pred.reshape(-1, 5)
                    use[1] = use[1].reshape(-1, 5)
                    assert pred.shape == use[1].shape, 'unequal shape error'
                    r2 = rsquared(pred[idx], use[1][idx])
            if r1 > r_1:
                r_1 = r1
                scale_1 = scale
            if r2 > r_2:
                r_2 = r2
                scale_2 = scale
    R1.append(r_1)
    R2.append(r_2)
    scale_1.append(scale_1)
    scale_2.append(scale_2)

assert len(R1)==5, 'index error'

print(R1)
print(R2)
print(scale_1)
print(scale_2)
